In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize


    

In [5]:
def sentenceSplit(rawText, library = "NLTK"):
    """
    Returns a list of sentences split using the specified library

    @type  rawText: string
    @param rawText: The text string to be seperated into sentences.
    @type  library: string
    @param library: The library to be used to split the sentences

    @rtype:   list
    @return:  list of split sentences returned from 
    """
    sentences = []
    try:
        if library == "NLTK":
            sentences = sent_tokenize(rawText)
        #TODO add in Spacy library
        
        return sentences
    
    except:
        return None
    
    raise Exception("Invalid library condition set")
    


def wordSplit(rawText, library = "NLTK"):
    """
    Returns a list of tokenized words split using the specified library

    @type  rawText: string
    @param rawText: The text string to be seperated into words.
    @type  library: string
    @param library: The library to be used to split the sentences

    @rtype:   list
    @return:  list of split sentences returned from 
    """
    sentences = []
    try:
        if library == "NLTK":
            sentences = sent_tokenize(rawText)
        #TODO add in Spacy library
        
        return sentences
    
    except:
        return None
    
    raise Exception("Invalid library condition set")

['This is sentence 1.', 'And This is sentence 2.']